In [1]:
# should use a vcf with SNPs but not with heavy filters. 
# only biallelic might speed things up, but do I miss out on important loci?

````bash
bcftools view -V indels,mnps,ref,bnd,other 1011Matrix.gvcf.gz | bcftools view -s AAA | bcftools view -i "AC>0 & FORMAT/DP > 10" | bcftools query -f "[%AD]\n" | gzip > 1011Matrix.gvcf.gz-AAA.AD.csv.gz
````

In [2]:
source("../BrusselSprouts/scripts/functions.R")


Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [ ]:
# out paste(args[1],".polyploid.score",sep="") <- folder schould be AD

In [42]:
samples=readLines("~/data/TRD/1011Matrix.gvcf.gz.samples")
length(samples)

[1] 1011

In [43]:
initial_timedate=Sys.time()
scripts_dir="/home/jnrunge/data/TRD/AD/scripts/"
jobname="Scev-ploidy"
sbatch_list=paste(scripts_dir,jobname,"-sbatch.list", sep="")
print(sbatch_list)
if(file.exists(sbatch_list)){
    system(command=paste("rm -f ", sbatch_list, sep=""))
}

for(s in samples){
    
    if(!file.exists(paste("~/data/TRD/AD/",s,".AD.csv.gz.polyploid.score",sep="")))
    {
        sbatch=paste("#!/bin/bash
#SBATCH -p fast          # The account name for the job
#SBATCH --job-name=",jobname,"  # The job name
#SBATCH -o ",scripts_dir,"logs/",jobname,"-",s,".out
#SBATCH -e ",scripts_dir,"logs/",jobname,"-",s,".err
#SBATCH -c 1                 # The number of cpu cores to use
#SBATCH --time=11:59:00       # The time the job will take to run 
#SBATCH --mem=4gb

date
source ~/anaconda3/etc/profile.d/conda.sh
conda activate bwaetc

cd ~/data/TRD/

bcftools view -V indels,mnps,ref,bnd,other 1011Matrix.gvcf.gz | bcftools view -s ",s," | bcftools view -i 'AC>0 & FORMAT/DP > 10' | bcftools query -f '[%AD]\\n' | gzip > AD/",s,".AD.csv.gz

conda activate JupyteR4

Rscript ~/TRD/scripts/polyploidy.R AD/",s,".AD.csv.gz AD/",s,".AD.csv.gz.polyploid.score

conda activate JupyteR4
Rscript ~/BrusselSprouts/scripts/Execute_Sbatches.R '",initial_timedate,"' ",sbatch_list," 20 ",jobname,"

date
",sep="")
        sbatch_file=paste("~/data/bruxellensis/mapping/scripts/",jobname,"-",s,".sbatch",sep="")
        #print(sbatch_file)
        writeLines(sbatch, sbatch_file)

        write(sbatch_file,file=sbatch_list,append=TRUE)


            
        }
}


print(system(command=paste("wc -l ", sbatch_list, sep=""),intern=TRUE))

#print(system(command=paste("Rscript ~/BrusselSprouts/scripts/Execute_Sbatches.R '",initial_timedate,"' ",sbatch_list," 20 ",jobname, sep=""), intern=TRUE))

[1] "/home/jnrunge/data/TRD/AD/scripts/Scev-ploidy-sbatch.list"
[1] "1011 /home/jnrunge/data/TRD/AD/scripts/Scev-ploidy-sbatch.list"
